<a href="https://colab.research.google.com/github/MMBAYAR/GUZ-24_ISLE421/blob/main/ISLE421(2)_Ders4(Tamamland%C4%B1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 49.9 MB/s eta 0:00:00


In [2]:
from gurobipy import*

In [22]:
#  Veri
#    Satış fiyatları
fiyat = [(70,60,50),
         (70,60,50),
         (70,60,50)]

hmmal = [(45,45,45),
         (35,35,35),
         (25,25,25)]

üsmal = [(4,4,4),
         (4,4,4),
         (4,4,4)]

rhmal = [1,1,1]

#...
#   Tedarik kısıtları
tedsağ = [5000,5000,5000]
rafkapsağ = [14000]
talepsağ = [3000,2000,1000]

tedsol = [
          [(1,1,1),
          (0,0,0),
          (0,0,0)],
          [(0,0,0),
           (1,1,1),
           (0,0,0)],
          [(0,0,0),
           (0,0,0),
           (1,1,1)]
        ]

rafkapsol = [(1,1,1),
             (1,1,1),
             (1,1,1)]

tlpsol = [
          [(1,0,0),
          (1,0,0),
          (1,0,0,)],
          [(0,1,0),
           (0,1,0),
           (0,1,0)],
          [(0,0,1),
           (0,0,1),
           (0,0,1)]
        ]

tlp_y = [10,10,10]

oktan_sol_pay = [
    [(12,0,0),
     (6,0,0),
     (8,0,0)],
    [(0,12,0),
     (0,6,0),
     (0,8,0)],
    [(0,0,12),
     (0,0,6),
     (0,0,8)]
]

oktan_sol_payda = [
    [(10,0,0),
     (10,0,0),
     (10,0,0)],
    [(0,8,0),
     (0,8,0),
     (0,8,0)],
    [(0,0,6),
     (0,0,6),
     (0,0,6)]
]

kükürt_sol_pay = [
    [(0.005,0,0),
     (0.02,0,0),
     (0.03,0,0)],
    [(0,0.005,0),
     (0,0.02,0),
     (0,0.03,0)],
    [(0,0,0.005),
     (0,0,0.02),
     (0,0,0.03)]
]

kükürt_sol_payda = [
    [(0.01,0,0),
     (0.01,0,0),
     (0.01,0,0)],
    [(0,0.005,0),
     (0,0.005,0),
     (0,0.005,0)],
    [(0,0,0.01),
     (0,0,0.01),
     (0,0,0.01)]
]

oktan_sağ = [0,0,0]

kükürt_sağ = [0,0,0]

In [23]:
len(tedsol)

3

In [24]:
m = Model()

#  Karar değişkenleri
x = m.addVars(len(fiyat),                   #  i'lerimi izlemek için fiyat listesindeki demet sayısını aldım (satır sayısını, hm sayısını)
              len(fiyat[0]), vtype = 'c')   #  j'lerimi izlemek için fiyat listesindeki herhangi demetin eleman sayısını aldım (sütün sayısını, ürün sayısını)

y = m.addVars(len(fiyat[0]), vtype = 'c')   #  j'lerimi ...

#  Amaç fonksiyonu
#hasılat = quicksum( fiyat[i][j]*x[i,j] for (i,j) in x.keys() )
hasılat = quicksum( fiyat[i][j]*x[i,j] for i in range(len(fiyat)) for j in range(len(fiyat[0])) )
hm_alım_maliyeti = quicksum( hmmal[i][j]*x[i,j] for i in range(len(fiyat)) for j in range(len(fiyat[0])) )
üs_maliyeti = quicksum( üsmal[i][j]*x[i,j] for i in range(len(fiyat)) for j in range(len(fiyat[0])) )
reklam_maliyeti = quicksum( rhmal[j]*y[j] for j in range(len(fiyat[0])) )

m.setObjective( hasılat - hm_alım_maliyeti - üs_maliyeti - reklam_maliyeti , GRB.MAXIMIZE )

#  Kısıtlar
#    Tedarik Kısıtları
for k in range(len(tedsol)):
  m.addConstr(
  quicksum( tedsol[k][i][j]*x[i,j] for i in range(len(fiyat)) for j in range(len(fiyat[0])) ) <= tedsağ[k]
  )

#    Kapasite Kısıtı
m.addConstr(
quicksum( rafkapsol[i][j]*x[i,j] for i in range(len(fiyat)) for j in range(len(fiyat[0])) ) <= rafkapsağ[0]
)

#    Talep Kısıtları
for k in range(len(tedsol)):
  m.addConstr(
  quicksum( tlpsol[k][i][j]*x[i,j] for i in range(len(fiyat)) for j in range(len(fiyat[0])) ) <= talepsağ[k] + tlp_y[k]*y[k]
  )

#    Oktan Kıstları
for k in range(len(tedsol)):
  m.addConstr(
  quicksum( oktan_sol_pay[k][i][j]*x[i,j] for i in range(len(fiyat)) for j in range(len(fiyat[0])) )
  >=
  oktan_sağ[k] + quicksum( oktan_sol_payda[k][i][j]*x[i,j] for i in range(len(fiyat)) for j in range(len(fiyat[0])) )
  )

#    Kükürt Kıstları
for k in range(len(tedsol)):
  m.addConstr(
  quicksum( kükürt_sol_pay[k][i][j]*x[i,j] for i in range(len(fiyat)) for j in range(len(fiyat[0])) )
  <=
  kükürt_sağ[k] + quicksum( kükürt_sol_payda[k][i][j]*x[i,j] for i in range(len(fiyat)) for j in range(len(fiyat[0])) )
  )

#  Çözdürme
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 13 rows, 12 columns and 45 nonzeros
Model fingerprint: 0xc2ec5195
Coefficient statistics:
  Matrix range     [5e-03, 1e+01]
  Objective range  [1e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+04]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 9 rows, 10 columns, 31 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1140129e+05   2.421684e+03   0.000000e+00      0s
       4    1.8205000e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.820500000e+05
